In [1]:
# reference: https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle/notebook

# import pandas as pd

# train = pd.read_csv('data/train.csv')
# test = pd.read_csv('data/test.csv')

In [2]:
# train

In [3]:
# test

In [4]:
# from collections import OrderedDict
# label_meaning = OrderedDict()
# for idx, l in enumerate(train.author.unique()):
#     label_meaning[l] = idx
    
# print(label_meaning)

In [5]:
# import numpy as np
# y = np.array(list(map(label_meaning.get, train.author.values)))
# y

In [6]:
# X_raw = train.text

In [7]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(min_df=3,  max_features=None,
#                              strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
#                              ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
#                              stop_words = 'english')

# vectorizer.fit(X_raw)
# X_tfidf =  vectorizer.transform(X_raw) 

In [8]:
# X_tfidf

In [9]:
# columns_index = train['id']
# tfidf_Requirements_df = pd.DataFrame(X_tfidf.T.todense(), index=vectorizer.get_feature_names(), columns=columns_index)
# tfidf_Requirements_df

In [10]:
# result = sum(tfidf_Requirements_df.astype(bool).sum())
# result

### Loading Data

In [11]:
from utils import read_dataset

X_tfidf, y = read_dataset()

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_tfidf, y, random_state=255, test_size=0.1, shuffle=True, stratify=y)


### Basic Naive Bayes

In [13]:
from sklearn.naive_bayes import MultinomialNB

# Basic Naive Bayes Model
model = MultinomialNB()
model.fit(X_train, Y_train)
scores = model.predict_proba(X_test)


In [14]:
pred = scores.argmax(axis=1)

In [15]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [16]:
import numpy as np
def accuracy(pred, truth):
    return np.sum(pred == truth)/truth.shape[0]

In [17]:
basic_Naive_Bayes_logloss = multiclass_logloss(Y_test, scores)
basic_Naive_Bayes_accuracy = accuracy(pred, Y_test)
print("logloss: %0.3f " % basic_Naive_Bayes_logloss)
print("Accuracy: %0.3f " % accuracy(pred, Y_test))

logloss: 0.572 
Accuracy: 0.831 


### Grid Search

In [18]:
from sklearn.model_selection import GridSearchCV

nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(X_train, Y_train)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best score: -0.490
Best parameters set:
	nb__alpha: 0.1


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    1.5s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    1.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.5s finished


### Naive Bayes after Grid Search

In [19]:
model = MultinomialNB(alpha= 0.1)
model.fit(X_train, Y_train)
scores = model.predict_proba(X_test)
pred = scores.argmax(axis=1)


In [20]:
grid_Search_Naive_Bayes_logloss = multiclass_logloss(Y_test, scores)
grid_Search_Naive_Bayes_accuracy = accuracy(pred, Y_test)
print("logloss: %0.3f " % grid_Search_Naive_Bayes_logloss)
print("Accuracy: %0.3f " % grid_Search_Naive_Bayes_accuracy)

print("Logloss improved from %0.3f to %0.3f" % (basic_Naive_Bayes_logloss, grid_Search_Naive_Bayes_logloss))
print("Accuracy improved from %0.3f to %0.3f" % (basic_Naive_Bayes_accuracy, grid_Search_Naive_Bayes_accuracy))

logloss: 0.427 
Accuracy: 0.835 
Logloss improved from 0.572 to 0.427
Accuracy improved from 0.831 to 0.835
